In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as ur

In [6]:
!pip install yahoofinancials

  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15190 sha256=f90756ba52e6a66ea90fc0a145ce49d7b93e73c746d3b8064bc5bc893a0393ad
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
from yahoofinancials import YahooFinancials
ticker = 'MSFT'
#yahoo_financials = YahooFinancials(ticker)
#balanceSheetHistory_dt = yahoo_financials.get_financial_stmts('annual', 'balance')
#incomeStatementHistory_dt = yahoo_financials.get_financial_stmts('annual', 'income')
#cashflowStatementHistory_dt= yahoo_financials.get_financial_stmts('annual', 'cash')
#apple_earnings_data = yahoo_financials.get_stock_earnings_data()
#apple_net_income = yahoo_financials.get_net_income()
#historical_stock_prices = yahoo_financials.get_historical_price_data('2008-09-15', '2018-09-15', 'weekly')

In [34]:
from yahoofinancials import YahooFinancials
def generate_financial_data(ticker):
    yahoo_financials = YahooFinancials(ticker)
    balanceSheetHistory_dt = yahoo_financials.get_financial_stmts('annual', 'balance')
    incomeStatementHistory_dt = yahoo_financials.get_financial_stmts('annual', 'income')
    cashflowStatementHistory_dt= yahoo_financials.get_financial_stmts('annual', 'cash')
    for i in range(len(cashflowStatementHistory_dt['cashflowStatementHistory'][ticker])):
      if i ==0:
        cs_df1=pd.DataFrame(cashflowStatementHistory_dt['cashflowStatementHistory'][ticker][i])
      else:
        cs_df2=pd.DataFrame(cashflowStatementHistory_dt['cashflowStatementHistory'][ticker][i])
        cs_df1=pd.concat([cs_df1, cs_df2], axis=1)
    for i in range(len(balanceSheetHistory_dt['balanceSheetHistory'][ticker])):
      if i ==0:
        bs_df1=pd.DataFrame(balanceSheetHistory_dt['balanceSheetHistory'][ticker][i])
      else:
        bs_df2=pd.DataFrame(balanceSheetHistory_dt['balanceSheetHistory'][ticker][i])
        bs_df1=pd.concat([bs_df1, bs_df2], axis=1)
    for i in range(len(incomeStatementHistory_dt['incomeStatementHistory'][ticker])):
      if i ==0:
        is_df1=pd.DataFrame(incomeStatementHistory_dt['incomeStatementHistory'][ticker][i])
      else:
        is_df2=pd.DataFrame(incomeStatementHistory_dt['incomeStatementHistory'][ticker][i])
        is_df1=pd.concat([is_df1, is_df2], axis=1)
    if len(incomeStatementHistory_dt['incomeStatementHistory'][ticker])==0:
      is_df1=pd.DataFrame(columns=[])
    if len(balanceSheetHistory_dt['balanceSheetHistory'][ticker])==0:
      bs_df1=pd.DataFrame(columns=[])
    if len(cashflowStatementHistory_dt['cashflowStatementHistory'][ticker])==0:
      cs_df1=pd.DataFrame(columns=[])
    consolidated=pd.concat([is_df1.T,bs_df1.T],axis=1)
    final=pd.merge(consolidated, cs_df1.T, left_index=True, right_index=True).drop_duplicates()
    final = final.assign(ticker=ticker).reset_index()
    final = final.loc[:, ~final.columns.duplicated()]
    final_out=final
    final_out=spark.createDataFrame(final)
    #print("check final")
    return final_out

In [11]:
import pandas as pd
#def generate_similarity(target):
source_text="Microsoft"
database=pd.read_excel("Dictionary_Project.xlsx") 
database.columns

Index(['Symbol', 'Name', 'Last Sale', 'Net Change', '% Change', 'Market Cap',
       'Country', 'IPO Year', 'Volume', 'Sector', 'Industry', 'Industry_Key'],
      dtype='object')

In [20]:
!pip install fuzzywuzzy

In [21]:
from fuzzywuzzy import fuzz # visit https://github.com/seatgeek/fuzzywuzzy for more details
from fuzzywuzzy import process
source_text="Microsoft"
database=pd.read_excel("Dictionary_Project.xlsx") 
questions = database['Name'].values.tolist()
mathed_question, score = process.extractOne(source_text, questions, scorer=fuzz.token_set_ratio) # use process.extract(.. limits = 3) to get multiple close matches
if score > 75: # arbitrarily chosen 50 to exclude matches not relevant to the query
    matched_row = database.loc[database['Name'] == mathed_question,]
    Symbol = matched_row['Symbol'].values[0]
    Name = matched_row['Name'].values[0]
    Industry_Key = matched_row['Industry_Key'].values[0]
    print("Engine is Running for the specified company {},{},{}".format(Symbol,Name,Industry_Key))
else: 
    print("Sorry I didn't find anything relevant to your query!")

Engine is Running for the specified company MSFT,Microsoft Corporation ,United States: Technology, Computer Software: Prepackaged Software


In [22]:
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
embedder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
entire_lot=database['Industry_Key'].tolist()
embedder_src=embedder([Industry_Key])
embedder_tgt=embedder(entire_lot)
cosine_calc=cosine_similarity(embedder_src,embedder_tgt)
database=database.assign(cosine_calc=cosine_calc[0])
database["Rank"] = database["cosine_calc"].rank(ascending = False,pct=False,method="min")
companies_of_interest=database.query("Rank<=20")['Symbol'].tolist()

In [37]:
companies_of_interest=database['Symbol'].tolist()

In [2]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 49.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=bf03aba04f5a150859cb631676810dcf6710bce6a6e01ff27ac9ab6c7fc25b39
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [14]:
generate_financial_data(ticker).show(10,False)

+----------+-------------+----------------------+---------+-------------------------+------------------+-----------+---------------+----------------+---------------+----------------+-----------+---------------------------------+--------------------------+------------+---------------+----------+----------------------+-------------------+----------------------------+----------------------+--------------------------+------------+---------------+-----------+-----------+----------------------------+-----------+----------------+----------+------------+-------------------+--------------+-----------------+-----------+------------------+----------------+-----------+----------------------+----------------------+----------------+-----------------+--------------------+------------+------------------+-----------------------+------------+----------------------+-------------+-------------------+------------+--------------------------+-----------------+-------------------+-----------------+-----------

In [36]:
generate_financial_data('AACG')

DataFrame[index: string, costOfRevenue: double, discontinuedOperations: double, ebit: double, effectOfAccountingCharges: double, extraordinaryItems: double, grossProfit: double, incomeBeforeTax: double, incomeTaxExpense: double, interestExpense: double, minorityInterest: double, netIncome_x: double, netIncomeApplicableToCommonShares: double, netIncomeFromContinuingOps: double, nonRecurring: double, operatingIncome: double, otherItems: double, otherOperatingExpenses: double, researchDevelopment: double, sellingGeneralAdministrative: double, totalOperatingExpenses: double, totalOtherIncomeExpenseNet: double, totalRevenue: double, capitalSurplus: double, cash: double, commonStock: double, deferredLongTermAssetCharges: double, goodWill: double, intangibleAssets: double, longTermInvestments: double, netReceivables: double, netTangibleAssets: double, otherAssets: double, otherCurrentAssets: double, otherCurrentLiab: double, otherLiab: double, otherStockholderEquity: double, propertyPlantEqui

In [ ]:
a=generate_financial_data('AACG')
for ticker in companies_of_interest:
    try:
        b=generate_financial_data(ticker)
        #if b.count>0:
        c=a.unionByName(b, allowMissingColumns=True)
        a=c
    except (ValueError,TypeError,AttributeError,AssertionError):
        print("missed to load {}".format(ticker))
    print(a.count)

<bound method DataFrame.count of DataFrame[index: string, costOfRevenue: double, discontinuedOperations: double, ebit: double, effectOfAccountingCharges: double, extraordinaryItems: double, grossProfit: double, incomeBeforeTax: double, incomeTaxExpense: double, interestExpense: double, minorityInterest: double, netIncome_x: double, netIncomeApplicableToCommonShares: double, netIncomeFromContinuingOps: double, nonRecurring: double, operatingIncome: double, otherItems: double, otherOperatingExpenses: double, researchDevelopment: double, sellingGeneralAdministrative: double, totalOperatingExpenses: double, totalOtherIncomeExpenseNet: double, totalRevenue: double, capitalSurplus: double, cash: double, commonStock: double, deferredLongTermAssetCharges: double, goodWill: double, intangibleAssets: double, longTermInvestments: double, netReceivables: double, netTangibleAssets: double, otherAssets: double, otherCurrentAssets: double, otherCurrentLiab: double, otherLiab: double, otherStockholder

In [41]:
a.to_csv("export_data.csv")

In [42]:
b=generate_financial_data('AADI')

In [45]:
a=a.astype(str)
b=b.astype(str)

In [49]:
pd.concat([b,a],axis=0,sort=True)

ValueError: ignored

In [30]:
a

,index,costOfRevenue,discontinuedOperations,ebit,effectOfAccountingCharges,extraordinaryItems,grossProfit,incomeBeforeTax,incomeTaxExpense,interestExpense,minorityInterest,netIncome_x,netIncomeApplicableToCommonShares,netIncomeFromContinuingOps,nonRecurring,operatingIncome,otherItems,otherOperatingExpenses,researchDevelopment,sellingGeneralAdministrative,totalOperatingExpenses,totalOtherIncomeExpenseNet,totalRevenue,capitalSurplus,cash,commonStock,deferredLongTermAssetCharges,goodWill,intangibleAssets,longTermInvestments,minorityInterest,netReceivables,netTangibleAssets,otherAssets,otherCurrentAssets,otherCurrentLiab,otherLiab,otherStockholderEquity,propertyPlantEquipment,retainedEarnings,...,capitalSurplus,cash,commonStock,goodWill,intangibleAssets,inventory,netReceivables,netTangibleAssets,otherAssets,otherCurrentAssets,otherCurrentLiab,otherLiab,propertyPlantEquipment,retainedEarnings,totalAssets,totalCurrentAssets,totalCurrentLiabilities,totalLiab,totalStockholderEquity,shortTermInvestments,capitalExpenditures,changeInCash,changeToAccountReceivables,changeToInventory,changeToLiabilities,changeToNetincome,changeToOperatingActivities,depreciation,dividendsPaid,issuanceOfStock,netIncome_y,otherCashflowsFromInvestingActivities,repurchaseOfStock,totalCashFromFinancingActivities,totalCashFromOperatingActivities,totalCashflowsFromInvestingActivities,netBorrowings,otherCashflowsFromFinancingActivities,investments,ticker
0,2020-12-31,104425332.0,NaN,-104357949.0,NaN,NaN,57742215.0,-110853584.0,-10268836.0,NaN,49341369.0,-92198032.0,-98382604.0,-100584748.0,NaN,-104357949.0,NaN,-330224.0,8832488.0,153597900.0,266525496.0,-6495635.0,162167547.0,541272503.0,112723433.0,4716675.0,2491792.0,194754963.0,110586111.0,44000000.0,49341369.0,2274723.0,-106936211.0,25442056.0,5941444.0,227398942.0,28985472.0,-37424722.0,79898302.0,-298533669.0,...,5161000.0,79025000.0,209000.0,3229000.0,38000.0,82219000.0,52005000.0,347598000.0,579000.0,3263000.0,19159000.0,37510000.0,224911000.0,345495000.0,449008000.0,220251000.0,59033000.0,98143000.0,350865000.0,NaN,-67802000.0,37915000.0,19859000.0,-9726000.0,-1145000.0,19116000.0,-161000.0,25634000.0,-19815000.0,21418000.0,79009000.0,6469000.0,-31229000.0,-29626000.0,128814000.0,-61273000.0,NaN,NaN,NaN,AAON
1,2019-12-31,79345327.0,4894197.0,-108839996.0,NaN,NaN,18424840.0,-141261615.0,-7149119.0,NaN,50391547.0,-122253989.0,-128262480.0,-134112496.0,NaN,-108839996.0,NaN,-588147.0,11817255.0,116035728.0,206610163.0,-32421619.0,97770167.0,560814066.0,154197758.0,4692312.0,11464891.0,200478795.0,135599770.0,45726391.0,50391547.0,17633049.0,-35938492.0,27867641.0,11729344.0,210525913.0,48241809.0,-37478167.0,82857085.0,-200151065.0,...,3631000.0,26797000.0,208000.0,3229000.0,272000.0,73601000.0,68200000.0,286639000.0,597000.0,17576000.0,17633000.0,25256000.0,179777000.0,286301000.0,371424000.0,187549000.0,56028000.0,81284000.0,290140000.0,NaN,-37166000.0,42379000.0,-13412000.0,2557000.0,705000.0,19667000.0,6795000.0,22766000.0,-16645000.0,12625000.0,53711000.0,51000.0,-20793000.0,-18500000.0,97925000.0,-37046000.0,6614000.0,-301000.0,NaN,AAON
2,2018-12-31,4251451.0,918665587.0,-67791950.0,NaN,NaN,-2912859.0,-68053474.0,0.0,NaN,39576017.0,854925914.0,835591366.0,-68053474.0,NaN,-67791950.0,NaN,-3793418.0,19594484.0,49078025.0,69130542.0,-261524.0,1338592.0,410195990.0,190586342.0,3534871.0,NaN,NaN,17122578.0,66390898.0,39576017.0,18685634.0,258694261.0,799652.0,4122926.0,7623085.0,NaN,-38288364.0,37430741.0,-71888585.0,...,NaN,1994000.0,208000.0,3229000.0,506000.0,77612000.0,60006000.0,245708000.0,598000.0,NaN,13938000.0,11060000.0,163003000.0,249235000.0,307994000.0,140658000.0,47491000.0,58551000.0,249443000.0,NaN,-37268000.0,-19463000.0,-2832000.0,-5598000.0,-764000.0,10858000.0,-2344000.0,17655000.0,-16728000.0,4987000.0,42329000.0,53000.0,-27943000.0,-39684000.0,54856000.0,-34635000.0,6614000.0,-301000.0,8944000.0,AAON
3,2017-12-31,4984730.0,82437963.0,-59395194.0,NaN,NaN,2508887.0,-88022197.0,19920486.0,N

In [ ]:
a.to_csv("complete_data.csv")

In [ ]:
a["ticker"].unique()

In [ ]:
a=pd.concat([a,generate_financial_data(Symbol)])


In [ ]:
a["ticker"].unique()